In [2]:
import hashlib
import bcrypt

In [3]:
# SHA1
m=hashlib.sha1(b"Chulalongkorn").hexdigest()
print(m)

ca8a68498ae67cd14c15f5ebf043633224005759


In [4]:
# MD5
m=hashlib.md5(b"Chulalongkorn").hexdigest()
print(m)

46fa3b56c660faff420190c18c98a56b


In [5]:
# BCRYPT
salt = bcrypt.gensalt()
m=bcrypt.hashpw(b"Chulalongkorn", salt).decode()
print(m)

$2b$12$oXVg7I3As6eKOYQZtYTAjO7qd9DGs1zYXZxYQUEhK1THAMjN4QJM2


In [ ]:
# 1. Objective: Understand how attackers use pre-built word lists (dictionaries) to
# crack hashes of common passwords.
# Scenario: You have discovered a SHA-1 hash in a compromised system:
# d54cc1fe76f5186380a0939d2fc1723c44e8a5f7.
# You suspect the password is a simple, common word, possibly with some
# character substitutions.
# Task: Write a Python program that reads a list of words, applies common
# substitutions, hashes the result, and checks if it matches the target hash.
# Note that you might want to include substitution in your code (lowercase,
# uppercase, number for letter [‘o’ => 0 , ‘l’ => 1, ‘i’ => 1]).

with open("most-common.txt", "r") as file:
    common_passwords = file.read().splitlines()

target = 'd54cc1fe76f5186380a0939d2fc1723c44e8a5f7'
subs = {'o': '0', 'l': '1', 'i': '1', 'a': '4', 'e': '3', 'g': '6', 's': '5', 't': '7', 'b': '8', 'z': '2'}

def generate_variations(word, idx=0, current=""):
    if idx == len(word):
        return [current]
    c = word[idx]
    lower = c.lower()
    variations = [c, c.upper()] if c.isalpha() else [c]
    if lower in subs:
        variations.append(subs[lower])
        if subs[lower].isalpha():
            variations.append(subs[lower].upper())
    results = []
    for v in variations:
        results += generate_variations(word, idx+1, current+v)
    return results

for password in common_passwords:

    variations = generate_variations(password)
    # print(variations)
    
    for variation in variations:
        if hashlib.sha1(variation.encode()).hexdigest() == target:
            print(f"Found matching password: {variation}")
            break

Found matching password: ThaiLanD


In [ ]:
# 2. Objective: To understand why modern password hashing algorithms like
# bcrypt are more secure than older ones like MD5 and SHA-1.
# Task: Design and run an experiment to measure how many hashes each
# algorithm can compute in a fixed amount of time. The code must test atleast
# MD5 , SHA-1 , and bcrypt .
# (You may also try additional algorithms like SHA256, SHA512, scrypt,
# Argon2.)

import time

with open("most-common.txt", "r") as file:
    common_passwords = file.read().splitlines()

time_limit = 1
target = 'password'

hashing_algorithms = {
    "MD5": hashlib.md5,
    "SHA-1": hashlib.sha1,
    "bcrypt": bcrypt.hashpw,
}

hash_rate = {
    "MD5": 0,
    "SHA-1": 0,
    "bcrypt": 0,
}

for algo_name, algo_func in hashing_algorithms.items():
    start_time = time.time()
    count = 0
    while time.time() < start_time + time_limit:
        if algo_name == "bcrypt":
            salt = bcrypt.gensalt()
            algo_func(target.encode(), salt)
            hash_rate[algo_name] += 1
        else:
            algo_func(target.encode())
            hash_rate[algo_name] += 1
    print(f"{algo_name}: {hash_rate[algo_name]} hashes computed in {time_limit} seconds ({time_limit / hash_rate[algo_name]} seconds/hash)")


MD5: 2196773 hashes computed in 1 seconds (4.5521316949907887e-07 seconds/hash)
SHA-1: 2270678 hashes computed in 1 seconds (4.4039709725465257e-07 seconds/hash)
bcrypt: 5 hashes computed in 1 seconds (0.2 seconds/hash)


In [35]:
# 3. Objective: To apply the performance measurements to understand the
# importance of password length and algorithm choice.
# Task: Based on the measurements from Exercise 2, estimate how long it would
# take an attacker to brute-force a password of a given length
# You may assume that the password contains only upper-case, lower-case,
# numbers and symbols.
# What does it suggest about the length of a proper password. (ie. Use more
# than a year to brute force.)

charset_size = 26 + 26 + 10 + 32  # Uppercase + Lowercase + Digits + Symbols

for algo, rate in hash_rate.items():
    for i in range(1, 12):
        password_length = i
        total_combinations = charset_size ** password_length
        seconds = total_combinations / rate
        years = seconds / (60 * 60 * 24 * 365.25)
        if years > 1:
            print(f"Estimated time to brute-force a {password_length}-character password with {algo}: {years:.2f} years")
            break


Estimated time to brute-force a 8-character password with MD5: 87.93 years
Estimated time to brute-force a 8-character password with SHA-1: 85.07 years
Estimated time to brute-force a 5-character password with bcrypt: 46.51 years


In [ ]:
# 4. If a given hash value is from a bcrypt algorithm, is it practical to do a brute-force attack?
# No, because it will take too much time.

In [ ]:
# 5. If a given hash value is from a bcrypt algorithm, is it practical to perform a rainbow table attack?
# No, because bcrypt use a unique salt for each password, making precomputed rainbow table useless.


In [ ]:
# 6. You have to store a password in a database. Please explain your design/strategy for securely storing it.
# (Hints: Proper hash function, Salting, Cost factor, Database Security)

# - Use hashing algorithm such as bcrypt to hash passwords before storing them.
# - Use a unique salt for each password to prevent rainbow table attacks.
# - Use a high cost factor such as bcrypt's rounds to make hashing slow to prevent brute-force attacks
# - Use a secure database to store hashed passwords with limited access and encryption.